In [1]:
#载入MNIST数据集，并创建Interactive Session
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
mnist = input_data.read_data_sets("MNIST_data/",one_hot = True)
sess = tf.InteractiveSession()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
#先定义好初始化函数
def weight_variable(shape):
    initial = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1,shape = shape)
    return tf.Variable(initial)

#定义好卷积层、池化层
def conv2d(x,w):
    return tf.nn.conv2d(x,w,strides=[1,1,1,1],padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],
                         padding = 'SAME')

In [3]:
#先定义输入的placeholder，x是特征，y_是真实的label
x = tf.placeholder(tf.float32,[None,784])
y_ = tf.placeholder(tf.float32,[None,10])
x_image = tf.reshape(x,[-1,28,28,1])

In [4]:
#接下来定义第一个卷积层
#先初始化，然后使用conv2d函数进行卷积，并加上偏置，再使用ReLU激活函数进行非线性处理
#最后使用最大池化函数对卷积的输出结果进行池化操作
w_conv1 = weight_variable([5,5,1,32])
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x_image,w_conv1)+b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [5]:
#定义第二个卷积层，卷积核数量变成了64
w_conv2 = weight_variable([5,5,32,64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1,w_conv2)+b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [6]:
#使用tf.reshape函数对第二个卷积层的输出tensor进行变形，将其转换为1D的向量
#然后连接一个全连接层，隐含节点为1024，并使用ReLU激活函数
w_fc1 = weight_variable([7*7*64,1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2,[-1,7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,w_fc1)+b_fc1)

In [7]:
#为减轻过拟合，下面使用一个Dropout层
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob)

In [8]:
#最后将Dropout层的输出连接一个Softmax层，得到最后的概率输出
w_fc2 = weight_variable([1024,10])
b_fc2 = bias_variable([10])
y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop,w_fc2)+b_fc2)

In [9]:
#定义损失函数
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_*tf.log(y_conv),
                                              reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

In [10]:
#再继续定义评测准确率的操作
correct_prediction = tf.equal(tf.argmax(y_conv,1),tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [11]:
#开始训练
tf.global_variables_initializer().run()
for i in range(20000):
    batch = mnist.train.next_batch(50)
    if i%100==0:
        train_accuracy = accuracy.eval(feed_dict = {x:batch[0],
                                                   y_:batch[1],
                                                   keep_prob:1.0})
        print("step %d, training accuracy %g"%(i,train_accuracy))
    train_step.run(feed_dict={x:batch[0],y_:batch[1],keep_prob:0.5})

step 0, training accuracy 0.04
step 100, training accuracy 0.86
step 200, training accuracy 0.9
step 300, training accuracy 0.88
step 400, training accuracy 0.94
step 500, training accuracy 0.94
step 600, training accuracy 0.92
step 700, training accuracy 0.96
step 800, training accuracy 1
step 900, training accuracy 1
step 1000, training accuracy 0.94
step 1100, training accuracy 0.98
step 1200, training accuracy 0.98
step 1300, training accuracy 0.92
step 1400, training accuracy 0.98
step 1500, training accuracy 1
step 1600, training accuracy 0.94
step 1700, training accuracy 1
step 1800, training accuracy 1
step 1900, training accuracy 0.98
step 2000, training accuracy 0.96
step 2100, training accuracy 0.96
step 2200, training accuracy 0.98
step 2300, training accuracy 0.94
step 2400, training accuracy 1
step 2500, training accuracy 0.94
step 2600, training accuracy 0.96
step 2700, training accuracy 0.98
step 2800, training accuracy 0.98
step 2900, training accuracy 0.98
step 3000, 

In [12]:
#训练完成后在最终的测试集上进行全面测试，得到整体的分类准确率
print("test accuracy %g"%accuracy.eval(feed_dict={x:mnist.test.images,
                                                 y_:mnist.test.labels,
                                                 keep_prob:1.0}))

test accuracy 0.9926


In [31]:
import pandas as pd
import numpy as np
X_test = pd.read_csv("tests.csv")
X_test = np.array(X_test).reshape(-1,28*28*1)

def minibatches(inputs=None, batch_size=None, shuffle=False):
    #assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batch_size + 1, batch_size):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batch_size]
        else:
            excerpt = slice(start_idx, start_idx + batch_size)
        yield inputs[excerpt]
predictions = []
for x_test in minibatches(X_test,50,False):
    lis =np.argmax(sess.run(y_conv, feed_dict={x: x_test,keep_prob:1.0}), 1)
    predictions = predictions+list(lis)

#保存结果  
submission = pd.DataFrame({
    "Label": predictions
})  
submission.to_csv("digits-submission.csv", index=True) 
print('Save successful!')

Save successful!


In [42]:
from skimage import io,transform
import glob
import os
#读取图片
path = "./imgs"
def read_img(path):
    imgs=[]
    for im in glob.glob(path+'/*.bmp'):
        try:
            print('reading the images:%s'%(im))
            img=io.imread(im)
            img=transform.resize(img,(28,28))
            imgs.append(img)
        except:
            print('WRONG:'+im)
    return np.asarray(imgs,np.float32).reshape(-1,784)
data=read_img(path)
predictions = np.argmax(sess.run(y_conv, feed_dict={x: data,keep_prob:1.0}), 1)
for i in range(len(predictions)):
    new = path+"/"+str(predictions[i])+"/"+str(i)+".bmp"
    io.imsave(new,data[i].reshape(28,28))

reading the images:./imgs\0.bmp
reading the images:./imgs\1.bmp
reading the images:./imgs\10.bmp
reading the images:./imgs\100.bmp
reading the images:./imgs\1000.bmp
reading the images:./imgs\1001.bmp
reading the images:./imgs\1002.bmp
reading the images:./imgs\1003.bmp
reading the images:./imgs\1004.bmp
reading the images:./imgs\1005.bmp
reading the images:./imgs\1006.bmp
reading the images:./imgs\1007.bmp
reading the images:./imgs\101.bmp
reading the images:./imgs\1010.bmp
reading the images:./imgs\1011.bmp
reading the images:./imgs\1012.bmp
reading the images:./imgs\1013.bmp
reading the images:./imgs\1014.bmp
reading the images:./imgs\1015.bmp
reading the images:./imgs\1016.bmp
reading the images:./imgs\1017.bmp
reading the images:./imgs\102.bmp
reading the images:./imgs\1020.bmp
reading the images:./imgs\1021.bmp
reading the images:./imgs\1022.bmp
reading the images:./imgs\1023.bmp
reading the images:./imgs\1024.bmp
reading the images:./imgs\1025.bmp
reading the images:./imgs\1026.

C:\Program Files\Anaconda3\envs\tensorflow-gpu\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Program Files\Anaconda3\envs\tensorflow-gpu\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Program Files\Anaconda3\envs\tensorflow-gpu\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Program Files\Anaconda3\envs\tensorflow-gpu\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', wi

reading the images:./imgs\1066.bmp
reading the images:./imgs\1067.bmp
reading the images:./imgs\107.bmp
reading the images:./imgs\1070.bmp
reading the images:./imgs\1071.bmp
reading the images:./imgs\1072.bmp
reading the images:./imgs\1073.bmp
reading the images:./imgs\1074.bmp
reading the images:./imgs\1075.bmp
reading the images:./imgs\1076.bmp
reading the images:./imgs\1077.bmp
reading the images:./imgs\11.bmp
reading the images:./imgs\110.bmp
reading the images:./imgs\1100.bmp
reading the images:./imgs\1101.bmp
reading the images:./imgs\1102.bmp
reading the images:./imgs\1103.bmp
reading the images:./imgs\1104.bmp
reading the images:./imgs\1105.bmp
reading the images:./imgs\1106.bmp
reading the images:./imgs\1107.bmp
reading the images:./imgs\111.bmp
reading the images:./imgs\1110.bmp
reading the images:./imgs\1111.bmp
reading the images:./imgs\1112.bmp
reading the images:./imgs\1113.bmp
reading the images:./imgs\1114.bmp
reading the images:./imgs\1115.bmp
reading the images:./imgs

C:\Program Files\Anaconda3\envs\tensorflow-gpu\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Program Files\Anaconda3\envs\tensorflow-gpu\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Program Files\Anaconda3\envs\tensorflow-gpu\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Program Files\Anaconda3\envs\tensorflow-gpu\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', wi

reading the images:./imgs\1147.bmp
reading the images:./imgs\115.bmp
reading the images:./imgs\1150.bmp
reading the images:./imgs\1151.bmp
reading the images:./imgs\1152.bmp
reading the images:./imgs\1153.bmp
reading the images:./imgs\1154.bmp
reading the images:./imgs\1155.bmp
reading the images:./imgs\1156.bmp
reading the images:./imgs\1157.bmp
reading the images:./imgs\116.bmp
reading the images:./imgs\1160.bmp
reading the images:./imgs\1161.bmp
reading the images:./imgs\1162.bmp
reading the images:./imgs\1163.bmp
reading the images:./imgs\1164.bmp
reading the images:./imgs\1165.bmp
reading the images:./imgs\1166.bmp
reading the images:./imgs\1167.bmp
reading the images:./imgs\117.bmp
reading the images:./imgs\1170.bmp
reading the images:./imgs\1171.bmp
reading the images:./imgs\1172.bmp
reading the images:./imgs\1173.bmp
reading the images:./imgs\1174.bmp
reading the images:./imgs\1175.bmp
reading the images:./imgs\1176.bmp
reading the images:./imgs\1177.bmp
reading the images:./im

...

reading the images:./imgs\830.bmp
reading the images:./imgs\831.bmp
reading the images:./imgs\832.bmp
reading the images:./imgs\833.bmp
reading the images:./imgs\834.bmp
reading the images:./imgs\835.bmp
reading the images:./imgs\836.bmp
reading the images:./imgs\837.bmp
reading the images:./imgs\840.bmp
reading the images:./imgs\841.bmp
reading the images:./imgs\842.bmp
reading the images:./imgs\843.bmp
reading the images:./imgs\844.bmp
reading the images:./imgs\845.bmp
reading the images:./imgs\846.bmp
reading the images:./imgs\847.bmp
reading the images:./imgs\850.bmp
reading the images:./imgs\851.bmp
reading the images:./imgs\852.bmp
reading the images:./imgs\853.bmp
reading the images:./imgs\854.bmp
reading the images:./imgs\855.bmp
reading the images:./imgs\856.bmp
reading the images:./imgs\857.bmp
reading the images:./imgs\860.bmp
reading the images:./imgs\861.bmp
reading the images:./imgs\862.bmp
reading the images:./imgs\863.bmp
reading the images:./imgs\864.bmp
reading the im

C:\Program Files\Anaconda3\envs\tensorflow-gpu\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Program Files\Anaconda3\envs\tensorflow-gpu\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Program Files\Anaconda3\envs\tensorflow-gpu\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Program Files\Anaconda3\envs\tensorflow-gpu\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', wi

reading the images:./imgs\931.bmp
reading the images:./imgs\932.bmp
reading the images:./imgs\933.bmp
reading the images:./imgs\934.bmp
reading the images:./imgs\935.bmp
reading the images:./imgs\936.bmp
reading the images:./imgs\937.bmp
reading the images:./imgs\940.bmp
reading the images:./imgs\941.bmp
reading the images:./imgs\942.bmp
reading the images:./imgs\943.bmp
reading the images:./imgs\944.bmp
reading the images:./imgs\945.bmp
reading the images:./imgs\946.bmp
reading the images:./imgs\947.bmp
reading the images:./imgs\950.bmp
reading the images:./imgs\951.bmp
reading the images:./imgs\952.bmp
reading the images:./imgs\953.bmp
reading the images:./imgs\954.bmp
reading the images:./imgs\955.bmp
reading the images:./imgs\956.bmp
reading the images:./imgs\957.bmp
reading the images:./imgs\960.bmp
reading the images:./imgs\961.bmp
reading the images:./imgs\962.bmp
reading the images:./imgs\963.bmp
reading the images:./imgs\964.bmp
reading the images:./imgs\965.bmp
reading the im

C:\Program Files\Anaconda3\envs\tensorflow-gpu\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Program Files\Anaconda3\envs\tensorflow-gpu\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Program Files\Anaconda3\envs\tensorflow-gpu\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Program Files\Anaconda3\envs\tensorflow-gpu\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', wi

C:\Program Files\Anaconda3\envs\tensorflow-gpu\lib\site-packages\skimage\util\dtype.py:122: UserWarning: Possible precision loss when converting from float32 to uint8
  .format(dtypeobj_in, dtypeobj_out))
